In [1]:
!pip install pyspark

In [2]:
from pyspark.sql.types import StructType, StructField, DoubleType
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark import SparkConf, SparkContext


Setting up spark environment. Make any changes... I added a bunch of stuff ot try and get rdds going but nothing worked for me!
Fyi : to see spark on browser, just enter http://localhost:4040 into browser

In [3]:
from pyspark.sql import SparkSession
from pyspark import SparkConf

conf = SparkConf().setAppName("Movie Review Analysis") \
    .set("spark.sql.execution.arrow.enabled", "true") \
    .set("spark.python.worker.reuse", "true") \
    .set("spark.python.worker.timeout", "1800") \
    .set("spark.network.timeout", "1800") \
    .set("spark.python.daemon.module", "pyspark.daemon") \
    .set("spark.python.worker.memory", "4g") \
    .set("spark.executor.instances", "2") \
    .set("spark.driver.memory", "4g")\

# Create or get the existing Spark session with the provided configuration
spark = SparkSession.builder \
    .config(conf=conf) \
    .getOrCreate()


reading and "transformaing" data (this could be changed a ton so go ahead)

In [4]:
movieReview_df = spark.read.option("header", "true").csv("test.csv")


In [5]:
movieReview_df = movieReview_df.withColumnRenamed('class_index', 'rating') \
                               .withColumnRenamed('_c1', 'review_title') \
                               .withColumnRenamed('_c2', 'review_text')


In [6]:
movieReview_df.show()

+------+--------------------+--------------------+
|rating|        review_title|         review_text|
+------+--------------------+--------------------+
|     2|            Great CD|"My lovely Pat ha...|
|     2|One of the best g...|Despite the fact ...|
|     1|Batteries died wi...|I bought this cha...|
|     2|works fine, but M...|Check out Maha En...|
|     2|Great for the non...|Reviewed quite a ...|
|     1|DVD Player crappe...|I also began havi...|
|     1|      Incorrect Disc|I love the style ...|
|     1|DVD menu select p...|I cannot scroll t...|
|     2|Unique Weird Orie...|"Exotic tales of ...|
|     1|"Not an ""ultimat...|Firstly,I enjoyed...|
|     2|Great book for tr...|I currently live ...|
|     1|                Not!|If you want to li...|
|     1|     A complete Bust|"This game requir...|
|     2|TRULY MADE A DIFF...|I have been using...|
|     1|didn't run off of...|Was hoping that t...|
|     1|          Don't buy!|First of all, the...|
|     2|Simple, Durable, ...|Th

In [7]:
movieReview_df.printSchema()

root
 |-- rating: string (nullable = true)
 |-- review_title: string (nullable = true)
 |-- review_text: string (nullable = true)



In [8]:
if movieReview_df.count() > movieReview_df.dropDuplicates(['review_title', 'review_text']).count():
    print("duplicated detected")

duplicated detected


In [9]:
movieReview_df.count()

400000

In [10]:
movieReview_df = movieReview_df.dropDuplicates(['review_title','review_text'])
movieReview_df.count()

399996

In [11]:
movieReview_df.groupBy('rating').count().show()

+------+------+
|rating| count|
+------+------+
|     1|199996|
|     2|200000|
+------+------+



In [12]:
import gensim.parsing.preprocessing as gsp
from pyspark.sql.types import StringType
from gensim import utils

filters = [
    gsp.strip_tags,
    gsp.strip_punctuation,
    gsp.strip_multiple_whitespaces,
    gsp.strip_numeric,
    gsp.remove_stopwords,
    gsp.strip_short,
    gsp.stem_text

]

removed review titles somewhere, we should probably keep it in so we can add it to review text and ahve one long line of text

In [13]:
movieReview_df.show(2)

+------+------------+--------------------+
|rating|review_title|         review_text|
+------+------------+--------------------+
|     1|        NULL|I can't give you ...|
|     1|        NULL|I ordered a 12 fo...|
+------+------------+--------------------+
only showing top 2 rows



-- START OF TODO: MAKE THIS RDD STUFF ACTUALLY WORK!!! --------------
I am trying to make it so we do the majority of cleaning through rdds, just for assignment reasons
So if someone could please try to do that, even with the cleaning I did above!
For example, i think there is a function called "distinct" 

In [14]:
def clean_text(x):
    # Get the review title and review text
    title = x[1]
    review_text = x[2]

    # Concatenate the title and review text
    combined_text = title + " " + review_text  # Add a space between title and text

    # Convert to lowercase and clean the combined text
    combined_text = combined_text.lower()
    combined_text = utils.to_unicode(combined_text)

    # Apply filters to the combined text
    for f in filters:
        combined_text = f(combined_text)

    return (x[0], combined_text)  # Return the ID and the cleaned combined text

In [ ]:
#movieReview_df = movieReview_df.limit(10)
input_rdd = movieReview_df.rdd.map(lambda x : clean_text(x))
input_rdd = movieReview_df.rdd

idea: join review text to review title, instead of getting rid of review title

In [16]:
#input_rdd = input_rdd.repartition(100)  # Adjust the number of partitions as needed
#input_rdd = input_rdd.distinct()


In [17]:
#print(input_rdd.take(1))

In [18]:
#input_df = input_rdd.toDF(['rating', 'review_title','review_text'])
#input_df.show(5)

-- END OF TODO --

This is just me manually cleaning, but we should rlly do it using rdd and map functions if possible.

In [19]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

# Your original cleaning function, adapted to work as a UDF
def clean_text(review_text):
    # Handle NULL or missing text
    if review_text is None:
        return ""
    
    # Convert to lowercase
    review_text = review_text.lower()
    
    # Remove special characters, numbers, and excessive punctuation
    review_text = re.sub(r'[^\w\s]', '', review_text)
    
    # Remove extra spaces
    review_text = re.sub(r'\s+', ' ', review_text).strip()
    
    # Remove stop words (optional: using a stop words remover)
    stop_words = set(["the", "a", "an", "and", "is", "it", "to", "you", "for", "in", "of", "on", "that", "this"])
    review_text = ' '.join([word for word in review_text.split() if word not in stop_words])
    
    return review_text
# Register the UDF
clean_review_udf = udf(clean_text, StringType())

In [20]:
# Apply the cleaning function to the 'text_column' and create a new column with the cleaned text
df_cleaned = movieReview_df.withColumn('cleaned_review_text', clean_review_udf(movieReview_df['review_text']))

# Optionally, drop the original column and keep only the cleaned column
df_cleaned = movieReview_df.drop('text_column')

In [21]:
df_cleaned.show(1)

+------+------------+--------------------+
|rating|review_title|         review_text|
+------+------------+--------------------+
|     1|        NULL|I can't give you ...|
+------+------------+--------------------+
only showing top 1 row



In [22]:
print(movieReview_df.count())

399996


In [23]:
mrText_df = movieReview_df.drop('reivew_title')

In [ ]:
from pyspark.sql.functions import col, when

# Drop rows with missing values
mrText_df = mrText_df.dropna()

# (Optional: Limit for testing purposes)
mrText_df = mrText_df.limit(10000)  # Remove this after testing


# Convert 'rating' column to integer
mrText_df = mrText_df.withColumn("rating", col("rating").cast("int"))

# Map 'rating' values: 2 -> 1, 1 -> 0
mrText_df = mrText_df.withColumn("rating", when(col("rating") == 2, 1).when(col("rating") == 1, 0))

# Verify the transformation
print(mrText_df.count())  # Count the number of rows
mrText_df.show()          # Display the updated DataFrame


100000
+------+--------------------+--------------------+
|rating|        review_title|         review_text|
+------+--------------------+--------------------+
|     1|                   !|This is a great b...|
|     0|! Concise && ( no...|If you want to lo...|
|     0|      !!! URGHHH !!!|This book was BOR...|
|     1|                !!!!|What can i say? I...|
|     0|!!!Not enough to ...|Make sure you che...|
|     0|             """ """|"I like the song ...|
|     0|""" Smile"".........|"To those of us t...|
|     0|         """1"" star| because 1/5 of t...|
|     1|"""A Christmas St...|I absolutely love...|
|     0|"""A Hole Too Sma...|"The seat is beau...|
|     1|"""A MUST HAVE"" ...|I would call this...|
|     1|   """A Must Have"""|If you don't know...|
|     1|"""A Very Excitin...|Benno Thomas has ...|
|     1|"""ALFA & ZULU"" ...|"Thanks to K6YB f...|
|     0|         """Alive"""|Go P.O.D way to l...|
|     0|"""Along the stre...|This book came hi...|
|     0|"""American"" La

END OF CLEANING -- ON TO TRAINING

In [25]:
train_df, test_df = mrText_df.randomSplit([0.8, 0.2])

In [ ]:
import xgboost as xgb
import numpy as np
from sklearn.metrics import accuracy_score

In [27]:
from pyspark.ml.feature import Tokenizer, Word2Vec
from pyspark.ml import Pipeline

# Rename or drop the existing 'words' column if it exists
if "words" in train_df.columns:
    train_df = train_df.drop("words")
if "features" in train_df.columns:
    train_df = train_df.drop("features")
    

# Define tokenizer and Word2Vec
tokenizer = Tokenizer(inputCol="review_text", outputCol="words")  # Ensure no conflicts
w2v = Word2Vec(vectorSize=300, minCount=0, inputCol="words", outputCol="features")


In [28]:
dec2vec_pipeline = Pipeline(stages=[tokenizer, w2v])


In [29]:
doc2vec_model = dec2vec_pipeline.fit(train_df)

ConnectionRefusedError: [WinError 10061] No connection could be made because the target machine actively refused it

In [ ]:
train_df = doc2vec_model.transform(train_df)


never ended up getting the train_df, test_df transform functions running. 0 clue why

In [ ]:

test_df = doc2vec_model.transform(test_df)


In [ ]:
# Show a few rows from the train DataFrame
print("Few rows from train df:")
train_df.show(2)

Few rows from train df:
+------+--------------------+--------------------+--------------------+--------------------+
|rating|        review_title|         review_text|               words|            features|
+------+--------------------+--------------------+--------------------+--------------------+
|     0|! Concise && ( no...|If you want to lo...|[if, you, want, t...|[-0.0464008737425...|
|     0|      !!! URGHHH !!!|This book was BOR...|[this, book, was,...|[-0.0472781334912...|
+------+--------------------+--------------------+--------------------+--------------------+
only showing top 2 rows



In [ ]:
train_features = train_df.select("features").collect()
train_labels = train_df.select("rating").collect()
test_features = test_df.select("features").collect()
test_labels = test_df.select("rating").collect()

In [ ]:
X_train = np.asarray([v[0].toArray() for v in train_features])
Y_train = np.asarray([v[0] for v in train_labels])
X_test = np.asarray([v[0].toArray() for v in train_features])
Y_test = np.asarray([v[0] for v in train_labels])


In [ ]:
xgbClassifier = xgb.XGBClassifier(max_depth=10, seed = 18238, objective = 'multi:softmax', num_class=2)
model = xgbClassifier.fit(X_train, Y_train)
pred = model.predict(X_test)

In [ ]:
from sklearn.model_selection import cross_val_score
auc_scores = cross_val_score(model, X, Y, cv=5, scoring='roc_auc')
print("Cross-validated AUC scores:", auc_scores)
print("Mean AUC score:", auc_scores.mean())

AUC Score is: 1.0


MAJOR ISSUE HERE: we get an accuracy of 100% ... which is probably not possible.